## Initial Imports

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Creating Structured Output

In [3]:
from pydantic import BaseModel, Field
from typing import List

class SocialMediaPost(BaseModel):
    platform: str = Field(..., description="The social media platform where the post will be published (e.g., Twitter, LinkedIn).")
    content: str = Field(..., description="The content of the social media post, including any hashtags or mentions.")

class ContentOutput(BaseModel):
    article: str = Field(..., description="The article, formatted in markdown.")
    social_media_posts: List[SocialMediaPost] = Field(..., description="A list of social media posts related to the article.")

## Loading Tasks and Agents YAML files

In [4]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Importing CrewAI Tools

In [5]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

## Setup Multi LLM models

In [6]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'
groq_llm = "groq/llama-3.1-70b-versatile"

## Creating Crew, Agents, and Tasks

In [7]:
# Creating Agents
market_news_monitor_agent = Agent(
    config=agents_config['market_news_monitor_agent'],
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=groq_llm,
)

data_analyst_agent = Agent(
    config=agents_config['data_analyst_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
    llm=groq_llm,
)

content_creator_agent = Agent(
    config=agents_config['content_creator_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
)

quality_assurance_agent = Agent(
    config=agents_config['quality_assurance_agent'],
)

# Creating Tasks
monitor_financial_news_task = Task(
    config=tasks_config['monitor_financial_news'],
    agent=market_news_monitor_agent
)

analyze_market_data_task = Task(
    config=tasks_config['analyze_market_data'],
    agent=data_analyst_agent
)

create_content_task = Task(
    config=tasks_config['create_content'],
    agent=content_creator_agent,
    context=[monitor_financial_news_task, analyze_market_data_task]
)

quality_assurance_task = Task(
    config=tasks_config['quality_assurance'],
    agent=quality_assurance_agent,
    output_pydantic=ContentOutput
)

# Creating Crew
content_creation_crew = Crew(
    agents=[
        market_news_monitor_agent,
        data_analyst_agent,
        content_creator_agent,
        quality_assurance_agent
    ],
    tasks=[
        monitor_financial_news_task,
        analyze_market_data_task,
        create_content_task,
        quality_assurance_task
    ],
    verbose=True
)

## Kicking off the Crew

In [ ]:
result = content_creation_crew.kickoff(inputs={
  'subject': 'Inflation in the US and the impact on the stock market in 2024'
})

# Agent: Lead Market Analyst
## Task: Monitor and analyze the latest news and updates related to the financial markets, with a particular focus on Inflation in the US and the impact on the stock market in 2024. Identify and summarize the most relevant and impactful news items that could influence market trends or investor decisions. Utilize financial news APIs and real-time market data tools to gather up-to-date information. Focus on detecting trends, regulatory changes, or significant economic indicators that directly relate to Inflation in the US and the impact on the stock market in 2024.



# Agent: Lead Market Analyst
## Thought: To create a comprehensive report on the latest news and updates related to inflation in the US and its impact on the stock market in 2024, I need to gather information from reliable financial news sources and real-time market data. My first step will be to search the internet for recent news articles and updates on US inflation and its effects on the stoc



# Agent: Lead Market Analyst
## Final Answer: 
**Federal Reserve Recalibrates Monetary Policy as Inflation Recedes**

At its December 2024 meeting, the Federal Reserve cut interest rates by 0.25%, resulting in total rate cuts of 1.00% since September. The FOMC now projects just two interest rate cuts in 2025, compared to earlier projections of four rate cuts. Investors anticipate the Fed may pause interest rate cuts when it meets again in early 2025.

The Fed's primary purpose in raising rates and keeping them elevated was to combat the highest inflation since the early 1980s. At its peak, inflation, as measured by the Consumer Price Index (CPI), reached 9.1% for the 12 months ending in June 2022. The most recent CPI reading, for the 12 months ending in November 2024, showed inflation at a much improved 2.7%. However, over the most recent months, inflation moved modestly higher.

The Fed's decision to cut interest rates has been met with a negative reaction from equity markets, with 



# Agent: Chief Data Strategist
## Thought: Thought: I have gathered valuable information on the Federal Reserve's monetary policy decisions in 2024 and their impact on the stock market. The search results provide insights into the Fed's interest rate cuts, the target range for the federal funds rate, and the impact of these decisions on investors. However, to create a comprehensive analysis report, I need to analyze the data and identify key trends and patterns. 
Based on the search results, it appears that the Federal Reserve has been actively managing monetary policy to combat inflation and support economic growth. The Fed's decision to cut interest rates by 0.25% in December 2024, resulting in total rate cuts of 1.00% since September, suggests that the Fed is taking a cautious approach to managing the economy. The current inflation rate of 2.9% as of December 2024 is still above the Fed's target range, but the trend suggests that inflation is slowly decreasing.
The search results 

## Social Content

In [ ]:
import textwrap

posts = result.pydantic.dict()['social_media_posts']
for post in posts:
    platform = post['platform']
    content = post['content']
    print(platform)
    wrapped_content = textwrap.fill(content, width=50)
    print(wrapped_content)
    print('-' * 50)

## Blog Post

In [ ]:
from IPython.display import display, Markdown
display(Markdown(result.pydantic.dict()['article']))